In [1]:
import sys
sys.path.append('../')


In [2]:
from DPF import ShardedFilesDatasetConfig, DatasetReader

config = ShardedFilesDatasetConfig.from_path_and_columns(
    'example_video_dataset',
    video_name_col='video_name',
    text_col='caption'
)

reader = DatasetReader()
processor = reader.read_from_config(config)


/home/jovyan/.mlspace/envs/alisa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:00<00:00, 234.36it/s]


# Check dataset and it's info

In [4]:
processor.print_summary()


Dataset format: sharded_files
Path: example_video_dataset
Modalities: ['video', 'text']
Columns: 3
Total samples: 5


In [5]:
processor.df


,video_path,text,split_name
0,example_video_dataset/0/0.mp4,Businessman Rides In A Car,0
1,example_video_dataset/0/1.mp4,Workspace In Natural Light,0
2,example_video_dataset/0/2.mp4,Woman In Dress Walking In Tulip Field,0
3,example_video_dataset/0/3.mp4,Film Burns Overlay,0
4,example_video_dataset/0/4.mp4,Portrait Leader Of The Roman Army,0


# Running filters

## VideoInfoFilter

In [6]:
from DPF.filters.videos.info_filter import VideoInfoFilter

datafilter = VideoInfoFilter(workers=16) 
print(datafilter.result_columns) # prints list of colums that will be added

processor.apply_data_filter(datafilter)


['is_correct', 'error', 'width', 'height', 'fps', 'duration']


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 15.06it/s]


In [7]:
processor.df


,video_path,text,split_name,is_correct,error,width,height,fps,duration
0,example_video_dataset/0/0.mp4,Businessman Rides In A Car,0,True,None,1280,720,25.000000,14.2800
1,example_video_dataset/0/1.mp4,Workspace In Natural Light,0,True,None,1280,720,25.000000,10.0800
2,example_video_dataset/0/2.mp4,Woman In Dress Walking In Tulip Field,0,True,None,1280,720,25.000000,12.1200
3,example_video_dataset/0/3.mp4,Film Burns Overlay,0,True,None,1280,720,23.976024,24.5245
4,example_video_dataset/0/4.mp4,Portrait Leader Of The Roman Army,0,True,None,1280,720,29.970030,9.2092


## Farneback optical flow

In [8]:
from DPF.filters.videos.farneback_filter import GunnarFarnebackFilter

datafilter = GunnarFarnebackFilter(pass_frames=12, min_frame_size=512, num_passes=12, workers=8)
processor.apply_data_filter(datafilter)


100%|██████████| 5/5 [00:03<00:00,  1.38it/s]


In [9]:
processor.df['optical_flow_farneback']


0    4.601
1    0.832
2    7.847
3    0.018
4    3.187
Name: optical_flow_farneback, dtype: float32

## RAFT optical flow

In [15]:
from DPF.filters.videos.raft_filter import RAFTOpticalFlowFilter

datafilter = RAFTOpticalFlowFilter(pass_frames=12, min_frame_size=512, num_passes=12, workers=8)
processor.apply_data_filter(datafilter)


100%|██████████| 5/5 [00:04<00:00,  1.07it/s]


In [11]:
processor.df['optical_flow_raft']


0     8.898000
1     4.232000
2    19.886000
3    41.337002
4    25.343000
Name: optical_flow_raft, dtype: float32

## VideoLLaVA captioning

In [8]:
from DPF.filters.videos.video_llava_filter import VideoLLaVAFilter

datafilter = VideoLLaVAFilter(
    prompt="detailed_video",
    device="cuda:0",
    workers=16,
    batch_size=8
)

processor.apply_data_filter(datafilter)


[2024-03-29 15:16:16,768] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/jovyan/.mlspace/envs/llava_34b/lib/python3.11/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/jovyan/.mlspace/envs/llava_34b/lib/python3.11/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/home/jovyan/.mlspace/envs/llava_34b/lib/python3.11/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.

100%|██████████| 1/1 [00:15<00:00, 15.02s/it]


In [9]:
processor.df


,video_path,text,split_name,is_correct,error,width,height,fps,duration,caption Video-LLaVA-7B prompt detailed_video
0,example_video_dataset/0/0.mp4,Businessman Rides In A Car,0,True,None,1280,720,25.000000,14.2800,This video is a black and white portrait of a ...
1,example_video_dataset/0/1.mp4,Workspace In Natural Light,0,True,None,1280,720,25.000000,10.0800,The video begins with a close-up shot of a tab...
2,example_video_dataset/0/2.mp4,Woman In Dress Walking In Tulip Field,0,True,None,1280,720,25.000000,12.1200,The video starts with a woman walking through ...
3,example_video_dataset/0/3.mp4,Film Burns Overlay,0,True,None,1280,720,23.976024,24.5245,This video features a woman with long blonde h...
4,example_video_dataset/0/4.mp4,Portrait Leader Of The Roman Army,0,True,None,1280,720,29.970030,9.2092,"The video showcases a man wearing a large, orn..."


In [10]:
processor.df['caption Video-LLaVA-7B prompt detailed_video'].tolist()


["This video is a black and white portrait of a man sitting in the back of his car, dressed in formal attire. He is wearing a suit and tie, which is typical of the formal dress code. As he sits in his seat, he looks out of frame, indicating that he is either waiting for someone or simply enjoying the view outside. His gaze is directed towards the rearview mirror, suggesting that there might be something interesting happening outside the car. Overall, the video captures a moment of stillness and contemplation, with the man'taking his gazed fixed on something outside of view.",
 'The video begins with a close-up shot of a table with various items on it, including a laptop, a mouse, some books, flowers, vases, potted plants, cups, bowls, bottles, scissors, pens, an apple, notebooks, glasses, remote controls, candles and a cell phone. A wooden chair is also present in the scene. After the shot, the camera pans around the table, showing the items from different angles. Then, it cuts to a sh

## LLaVA 1.5 captioning

In [12]:
from DPF.filters.images.llava_captioning_filter import LLaVaCaptioningFilter
from DPF.filters.videos.image_filter_adapter import ImageFilterAdapter

datafilter = LLaVaCaptioningFilter(workers=8, prompt='short-video', batch_size=1, device='cuda:0')
video_adapter = ImageFilterAdapter(datafilter, 0.5, workers=8)

processor.apply_data_filter(video_adapter)


Describe this video very shortly in 1-2 short sentences. Describe what is happening in this video.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at openai/clip-vit-large-patch14-336 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.6.layer_norm2.weight', 'text_model.encoder.layers.6.mlp.fc1.bias', 'text_model.encoder.layers.3.self_attn.q_proj.weight', 'text_model.encoder.layers.0.layer_norm1.weight', 'text_model.encoder.layers.7.mlp.fc1.weight', 'text_model.encoder.layers.8.mlp.fc2.bias', 'text_model.encoder.layers.1.mlp.fc1.weight', 'text_model.encoder.layers.6.self_attn.q_proj.weight', 'text_model.encoder.layers.7.self_attn.q_proj.weight', 'text_model.encoder.layers.4.self_attn.v_proj.weight', 'text_model.encoder.layers.0.self_attn.v_proj.weight', 'text_model.encoder.layers.10.self_attn.v_proj.bias', 'text_model.encoder.layers.2.mlp.fc2.weight', 'text_model.encoder.layers.3.self_attn.k_proj.bias', 'text_model.encoder.layers.9.self_attn.k_proj.bias', 'text_model.encoder.layers.10.mlp.fc2.bias', 'text_model.encoder.layers.5.self_attn.q_proj.bias', 'text_model.encod

100%|██████████| 5/5 [00:05<00:00,  1.12s/it]


In [13]:
processor.df


,video_path,text,split_name,is_correct,error,width,height,fps,duration,mean_optical_flow_farneback,mean_optical_flow_raft,caption liuhaotian/llava-v1.5-13b prompt short-video
0,example_video_dataset/0/0.mp4,Businessman Rides In A Car,0,True,None,1280,720,25.000000,14.2800,7.428,20.997999,"A man in a suit is sitting in a car, looking o..."
1,example_video_dataset/0/1.mp4,Workspace In Natural Light,0,True,None,1280,720,25.000000,10.0800,5.926,38.886002,"The image features a white desk with a laptop,..."
2,example_video_dataset/0/2.mp4,Woman In Dress Walking In Tulip Field,0,True,None,1280,720,25.000000,12.1200,10.874,49.102001,"A woman is walking through a flower garden, ca..."
3,example_video_dataset/0/3.mp4,Film Burns Overlay,0,True,None,1280,720,23.976024,24.5245,0.194,169.417999,"The video is a colorful, abstract scene with a..."
4,example_video_dataset/0/4.mp4,Portrait Leader Of The Roman Army,0,True,None,1280,720,29.970030,9.2092,10.816,65.942001,A man wearing a helmet and a blue feather is s...


In [14]:
processor.df['caption liuhaotian/llava-v1.5-13b prompt short-video'].tolist()


['A man in a suit is sitting in a car, looking out the window.',
 'The image features a white desk with a laptop, a chair, and a vase. The desk is situated in front of a window, and the laptop is open, possibly in use.',
 'A woman is walking through a flower garden, carrying a hat.',
 'The video is a colorful, abstract scene with a blurry background and a bright yellow light in the foreground.',
 'A man wearing a helmet and a blue feather is standing in front of a blue sky.']

## LITA captioning

In [12]:
from DPF.filters.videos.lita_filter import LITAFilter

datafilter = LITAFilter(batch_size=8)

processor.apply_data_filter(datafilter)


[2024-05-22 00:27:18,567] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
/home/user/conda/envs/dpf/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:20<00:00,  6.91s/it]
Some weights of the model checkpoint at ./lita-vicuna-v1-3-13b-finetune were not used when initializing LitaLlamaForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.17.self_attn.out_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.22.mlp.fc2.bias', 'model.vision_tow

In [13]:
datafilter.result_columns


['caption lita-vicuna-v1-3-13b-finetune prompt detailed_video']

In [14]:
processor.df['caption lita-vicuna-v1-3-13b-finetune prompt detailed_video'].tolist()


["The image features a man wearing a suit and tie, sitting in a car and looking out the window. The scene appears to be set in the rain, as the man is seen through the rain-covered window. The man's attire and the car's interior suggest a professional or formal setting. The overall atmosphere of the image is somewhat mysterious and intriguing, as the man's gaze seems to be directed towards something beyond the viewer's perspective.",
 'The video is a white room setup with a desk and chair, featuring a laptop, keyboard, mouse, and a potted plant. The room appears to be dimly lit, giving it a cozy atmosphere. The focus of the video is a laptop screen, which is displaying a video of a woman talking. The setup seems to be staged for a video call or a virtual meeting. The room contains a few other items, such as a cup, a vase, and a remote, but they are not the main subjects of the video.',
 'The video is a short, whimsical scene featuring a woman walking through a field of flowers with a l

## PLLAVA captioning

In [3]:
from DPF.filters.videos.pllava_filter import Pllava13bFilter

datafilter = Pllava13bFilter(batch_size=5, num_frames=32, prompt='short')

processor.apply_data_filter(datafilter)

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
Loading checkpoint shards: 100%|██████████| 6/6 [00:01<00:00,  5.82it/s]
Some weights of PllavaForConditionalGeneration were not initialized from the model checkpoint at weights/pllava-13b and are newly initialized: ['language_model.lm_head.weight', 'language_model.model.embed_tokens.weight', 'language_model.model.layers.0.input_layernorm.weight', 'language_model.model.layers.0.mlp.down_proj.weight', 'language_model.model.layers.0.mlp.gate_proj.weight', 'language_model.model.layers.0.mlp.up_proj.weight', 'language_model.model.layers.0.post_attention_layernorm.weight', 'language_model.model.layers.0.self_attn.k_proj.weight', 'language_model.model.layers.0.self_attn.o_proj.weight', 'language_model.model.layers.0.self_attn.q_proj.weight', 'language_model.model.layers.0.self_attn.v_proj.weight', 'language_model.model.layers.1.input_layernorm.weight', 'language_model.model

Use lora
Lora Scaling: 0.03125
Finish use lora
Loading weight from weights/pllava-13b
<All keys matched successfully>


100%|██████████| 1/1 [00:15<00:00, 15.83s/it]


In [4]:
datafilter.result_columns

['caption pllava-13b prompt short']

In [6]:
processor.df['caption pllava-13b prompt short'].tolist()

['The image shows a man sitting in the back seat of a car at night.',
 'The image shows a modern workspace with a laptop on a desk.',
 'The image shows a woman walking through a field of tulips.',
 'The image is a black and white video with a blurry background.',
 'The image shows a man dressed in ancient Roman armor, standing against a clear blue sky.']

In [7]:
from DPF.filters.videos.pllava_filter import Pllava13bFilter

datafilter = Pllava13bFilter(batch_size=5, num_frames=32, prompt='short')


You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
Loading checkpoint shards: 100%|██████████| 6/6 [00:01<00:00,  4.22it/s]
Some weights of PllavaForConditionalGeneration were not initialized from the model checkpoint at weights/pllava-13b and are newly initialized: ['language_model.lm_head.weight', 'language_model.model.embed_tokens.weight', 'language_model.model.layers.0.input_layernorm.weight', 'language_model.model.layers.0.mlp.down_proj.weight', 'language_model.model.layers.0.mlp.gate_proj.weight', 'language_model.model.layers.0.mlp.up_proj.weight', 'language_model.model.layers.0.post_attention_layernorm.weight', 'language_model.model.layers.0.self_attn.k_proj.weight', 'language_model.model.layers.0.self_attn.o_proj.weight', 'language_model.model.layers.0.self_attn.q_proj.weight', 'language_model.model.layers.0.self_attn.v_proj.weight', 'language_model.model.layers.1.input_layernorm.weight', 'language_model.model

Use lora
Lora Scaling: 0.03125
Finish use lora
Loading weight from weights/pllava-13b
<All keys matched successfully>


In [8]:
%%time
processor.apply_data_filter(datafilter)

  0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true 

CPU times: user 7.73 s, sys: 4 s, total: 11.7 s
Wall time: 17 s
